In [33]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 4: Ceres Search ---</h2><p>"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the <a href="/2019/day/10">Ceres monitoring station</a>!</p>
<p>As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her <em>word search</em> (your puzzle input). She only has to find one word: <code>XMAS</code>.</p>
<p>This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of <code>XMAS</code> - you need to find <em>all of them</em>. Here are a few ways <code>XMAS</code> might appear, where irrelevant characters have been replaced with <code>.</code>:</p><p>
</p><pre><code>..X...
.SAMX.
.A..A.
XMAS.S
.X....
</code></pre>
<p>The actual word search will be full of letters instead. For example:</p>
<pre><code>MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
</code></pre>
<p>In this word search, <code>XMAS</code> occurs a total of <code><em>18</em></code> times; here's the same word search again, but where letters not involved in any <code>XMAS</code> have been replaced with <code>.</code>:</p>
<pre><code>....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
</code></pre>
<p>Take a look at the little Elf's word search. <em>How many times does <code>XMAS</code> appear?</em></p>
</article>


In [34]:
from pprint import pprint

from scipy import ndimage


tests = [
    {
        "name": "Small Example",
        "s": """
            ..X...
            .SAMX.
            .A..A.
            XMAS.S
            .X....
        """,
        "expected": 4,
    },
    {
        "name": "Example",
        "s": """
            MMMSXXMASM
            MSAMXMSMSA
            AMXSXMAAMM
            MSAMASMSMX
            XMASAMXAMM
            XXAMMXXAMA
            SMSMSASXSS
            SAXAMASAAA
            MAMMMXMMMM
            MXMXAXMASX
        """,
        "expected": 18,
    },
]


def count_XMAS(s: str) -> int:
    xmas, samx = np.array(list("XMAS")), np.array(list("SAMX"))
    m = np.array([list(l.strip()) for l in s.strip().splitlines()])
    # horizontal
    count = count_in_rows(xmas, samx, m)
    # vertical
    count += count_in_rows(xmas, samx, m.T)
    # diagonal
    count += count_in_diagonals(xmas, samx, m)
    # anti diagonal
    count += count_in_diagonals(xmas, samx, np.fliplr(m))

    return count


def count_in_diagonals(xmas, samx, m):
    rows, cols = m.shape
    width = xmas.size
    count = 0
    for offset in range(-rows + 1, cols):
        diag = m.diagonal(offset=offset)
        for c in range(width, diag.size + 1):
            ll = diag[c - width : c]
            if np.array_equal(ll, xmas) or np.array_equal(ll, samx):
                count += 1
    return count


def count_in_rows(xmas, samx, m) -> int:
    rows, cols = m.shape
    width = xmas.size
    count = 0
    for r in range(rows):
        for c in range(width, cols + 1):
            ll = m[r, c - width : c]
            if np.array_equal(ll, xmas) or np.array_equal(ll, samx):
                count += 1
    return count


@test(tests=tests)
def partI_test(s: str) -> int:
    return count_XMAS(s)


Test Small Example passed, for partI_test.
Test Example passed, for partI_test.
Success


In [35]:
with open("../input/day4.txt") as f:
    puzzle = f.read()

print(f"Part I: {count_XMAS(puzzle)}")

Part I: 2532


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2532</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Elf looks quizzically at you. Did you misunderstand the assignment?</p>
<p>Looking for the instructions, you flip over the word search to find that this isn't actually an <code><em>XMAS</em></code> puzzle; it's an <span title="This part originally involved searching for something else, but this joke was too dumb to pass up."><code><em>X-MAS</em></code></span> puzzle in which you're supposed to find two <code>MAS</code> in the shape of an <code>X</code>. One way to achieve that is like this:</p>
<pre><code>M.S
.A.
M.S
</code></pre>
<p>Irrelevant characters have again been replaced with <code>.</code> in the above diagram. Within the <code>X</code>, each <code>MAS</code> can be written forwards or backwards.</p>
<p>Here's the same example from before, but this time all of the <code>X-MAS</code>es have been kept instead:</p>
<pre><code>.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
</code></pre>
<p>In this example, an <code>X-MAS</code> appears <code><em>9</em></code> times.</p>
<p>Flip the word search from the instructions back over to the word search side and try again. <em>How many times does an <code>X-MAS</code> appear?</em></p>
</article>


In [36]:
from numpy.lib.stride_tricks import sliding_window_view


tests = [
    {
        "name": "Small Example",
        "s": """
            M.S
            .A.
            M.S
        """,
        "expected": 1,
    },
    {
        "name": "Example",
        "s": """
            MMMSXXMASM
            MSAMXMSMSA
            AMXSXMAAMM
            MSAMASMSMX
            XMASAMXAMM
            XXAMMXXAMA
            SMSMSASXSS
            SAXAMASAAA
            MAMMMXMMMM
            MXMXAXMASX
        """,
        "expected": 9,
    },
]


def count_X_MAS(s: str) -> int:
    m = np.array([list(l.strip()) for l in s.strip().splitlines()])
    mas, sam = np.array(list("MAS")), np.array(list("SAM"))

    count = 0
    windows = sliding_window_view(m, (3, 3))

    for i, j in product(range(windows.shape[0]), range(windows.shape[1])):
        w = windows[i, j]
        diag = np.diagonal(w, 0)
        anti = np.diagonal(np.fliplr(w), 0)

        if (np.array_equal(diag, mas) or np.array_equal(diag, sam)) and (
            np.array_equal(anti, mas) or np.array_equal(anti, sam)
        ):
            count += 1

    return count


@test(tests=tests)
def partI_test(s: str) -> int:
    return count_X_MAS(s)


Test Small Example passed, for partI_test.
Test Example passed, for partI_test.
Success


<link href="style.css" rel="stylesheet"></link>


In [37]:
print(f"Part II: {count_X_MAS(puzzle)}")

Part II: 1941


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1941</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
